# Faster fingerprint screening in rdkit
I recently played around with fingerprint based substructure screens.
I found that `DataStructs.AllProbeBitsMatch` is rather slow and that 
converting the fingerprint to a python integer (arbitrary size)
and using bitwise operations on it gives the same results in less time.
https://github.com/rdkit/rdkit/blob/master/Regress/Scripts/fingerprint_screenout.py

In [82]:
!wget -q -nc https://raw.githubusercontent.com/greglandrum/rdkit_blog/master/data/chembl21_25K.pairs.txt.gz 
!wget -q -nc https://raw.githubusercontent.com/greglandrum/rdkit_blog/master/data/zinc.frags.500.q.smi
!wget -q -nc https://raw.githubusercontent.com/greglandrum/rdkit_blog/master/data/zinc.leads.500.q.smi
!wget -q -nc https://raw.githubusercontent.com/greglandrum/rdkit_blog/master/data/fragqueries.q.txt
    

In [83]:
import time
import gzip
import rdkit
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
import os
rdkit_data = './'

In [84]:
# Get a list of test molecules
mols = []
with gzip.open(os.path.join(rdkit_data, 'chembl21_25K.pairs.txt.gz'), 'rb') as inf:
    for line in inf:
        line = line.decode().strip().split()
        smi1 = line[1]
        smi2 = line[3]
        mols.append(Chem.MolFromSmiles(smi1))
        mols.append(Chem.MolFromSmiles(smi2))

In [85]:
# Load test substructures
name2fn = {'frags': 'zinc.frags.500.q.smi',
           'leads': 'zinc.leads.500.q.smi',
           'pieces': 'fragqueries.q.txt'}

queries = {}
for name, fn in name2fn.items():
    with open(os.path.join(rdkit_data, fn), 'r') as f:
        smiles = [s.split('\t')[0] for s in f.read().split('\n')]
        ms = [Chem.MolFromSmiles(s) for s in smiles]
        print(name, len(smiles))
        queries[name] = ms

frags 501
leads 501
pieces 824


In [ ]:
# calculate fingerprints for molecules. these are ExplicitBitVect objects
mol_ebvs = [Chem.PatternFingerprint(m,2048) for m in mols]

In [11]:
# calculate fingerprints for substructures. these are ExplicitBitVect objects
name2ebvs = {}
for name, ms in queries.items():
    name2ebvs[name] = [Chem.PatternFingerprint(m, 2048) for m in ms]

In [13]:
# convert ebvs to python integers
def ebvs2int(ebvs):
    return [int(e.ToBitString(), base=2) for e in ebvs]

name2ints = {name: ebvs2int(ebvs) for name, ebvs in name2ebvs.items()}
mol_ints = ebvs2int(mol_ebvs)

## Full substructure search

In [29]:
from time import time
# DataStructs.AllProbeBitsMatch
results = []
for screen_only in [True, False]: # if true exact substructure match is skipped.
    for name, query_mols in queries.items(): 
        imatch = []
        jmatch = []
        t0 = time()
        query_ebvs = name2ebvs[name]
        for i, mol_ebv in enumerate(mol_ebvs):
            for j, query_ebv in enumerate(query_ebvs):
                if DataStructs.AllProbeBitsMatch(query_ebv, mol_ebv):
                    if screen_only or mols[i].HasSubstructMatch(query_mols[j]):
                        imatch.append(i)
                        jmatch.append(j)
        duration = time() - t0
        
        results.append({
            'screen_method': 'apbm',
            'screen_only': screen_only,
            'query_set': name,
            'matches': (imatch, jmatch),
            'duration': duration
        })

In [37]:
for screen_only in [True, False]:
    for name, query_mols in queries.items():
        imatch = []
        jmatch = []
        t0 = time()
        query_ints = name2ints[name]
        for i, mol_int in enumerate(mol_ints):
            for j, q_int in enumerate(query_ints):
                if (q_int & mol_int) == q_int:
                    if screen_only or mols[i].HasSubstructMatch(query_mols[j]):
                        imatch.append(i)
                        jmatch.append(j)
        duration = time() - t0
        
        results.append(
            {'screen_method': 'int',
             'screen_only': screen_only,
             'query_set': name,
             'matches': (imatch, jmatch),
             'duration': duration
            })

In [96]:
import pandas as pd


df_apbm = pd.DataFrame(results[:6]) # results for AllProbeBitsMatch
df_int = pd.DataFrame(results[6:]) # results for integer implementation
df = df_apbm.merge(df_int, on=('query_set', 'screen_only'), suffixes=('_apbm', '_int')) # merge

# check if results agree
for a,b in zip(df['matches_apbm'], df['matches_int']):
    assert a == b

df['speedup'] = df['duration_apbm'] / df['duration_int']
df[['screen_only', 'query_set', 'duration_apbm', 'duration_int', 'speedup']]

,screen_only,query_set,duration_apbm,duration_int,speedup
0,True,frags,16.812480,4.071757,4.129048
1,True,leads,17.210873,3.957238,4.349213
2,True,pieces,30.277018,8.124406,3.726675
3,False,frags,16.699876,4.255016,3.924751
4,False,leads,16.804592,4.111600,4.087117
5,False,pieces,45.135765,23.746885,1.900703


For the screen only this gives a speedup of about 4x.
If the exact matching is also included the speedup is still high showing that `AllProbeBitsMatch` takes up quite a lot of time.